<a href="https://colab.research.google.com/github/zar7364/DataScience/blob/main/Pyspark_Netflix_Recommendation_using_Big_Data_Analytic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Import modul and Create Session
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
appName="Sistem Rekomendasi Film"
spark = SparkSession.builder.master("local").appName(appName).getOrCreate()
sc=spark.sparkContext
sc


<SparkContext master=local appName=Sistem Rekomendasi Film>

In [6]:
#membuat data file ke DataFrame
ratings = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Recommendation Movie/ratings.csv", header=True, inferSchema=True)
movies = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Recommendation Movie/movies.csv", header=True, inferSchema=True)
df=ratings.join(movies,"movieId")
df.show(5)

+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|rating| timestamp|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|     31|     1|   2.5|1260759144|Dangerous Minds (...|               Drama|
|   1029|     1|   3.0|1260759179|        Dumbo (1941)|Animation|Childre...|
|   1061|     1|   3.0|1260759182|     Sleepers (1996)|            Thriller|
|   1129|     1|   2.0|1260759185|Escape from New Y...|Action|Adventure|...|
|   1172|     1|   4.0|1260759205|Cinema Paradiso (...|               Drama|
+-------+------+------+----------+--------------------+--------------------+
only showing top 5 rows



In [7]:
#Pick the colomns the we need
data=df.select("userId","movieId","rating")
data.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     31|   2.5|
|     1|   1029|   3.0|
|     1|   1061|   3.0|
|     1|   1129|   2.0|
|     1|   1172|   4.0|
+------+-------+------+
only showing top 5 rows



In [8]:
#Split data with ration 80% training and 20% testing
splits=data.randomSplit([0.8,0.2])
train=splits[0].withColumnRenamed("rating","label")
test=splits[1].withColumnRenamed("rating","trueLabel")

train_row=train.count()
test_row=test.count()


print("Total data train is :",train_row)
print("Total data test is:", test_row)

Total data train is : 80021
Total data test is: 19983


In [20]:
test.show()

+------+-------+---------+
|userId|movieId|trueLabel|
+------+-------+---------+
|     1|   1405|      1.0|
|     2|     17|      5.0|
|     2|     62|      3.0|
|     2|    110|      4.0|
|     2|    261|      4.0|
|     2|    349|      4.0|
|     2|    350|      4.0|
|     2|    357|      3.0|
|     2|    372|      3.0|
|     2|    382|      3.0|
|     2|    474|      2.0|
|     2|    485|      3.0|
|     2|    500|      4.0|
|     2|    527|      4.0|
|     2|    537|      4.0|
|     2|    552|      3.0|
|     2|    661|      4.0|
|     3|    267|      3.0|
|     3|    527|      3.0|
|     3|    592|      3.0|
+------+-------+---------+
only showing top 20 rows



In [10]:
#Modelling
als=ALS(maxIter=19,regParam=0.01, userCol="userId", itemCol="movieId",ratingCol="label")
model=als.fit(train)


In [18]:
#Let's Test and Evaluate
prediction = model.transform(test)
prediction.show(5)

+------+-------+---------+----------+
|userId|movieId|trueLabel|prediction|
+------+-------+---------+----------+
|   460|    471|      5.0| 3.2884648|
|   491|    471|      3.0|  5.311088|
|   309|    471|      4.0| 4.0655103|
|   241|    471|      4.0| 3.7564776|
|   624|    471|      4.0|  4.006094|
+------+-------+---------+----------+
only showing top 5 rows



In [11]:
#Let's Test and Evaluate
prediction = model.transform(test)
prediction.join(movies, "movieId").select(
    "userId", "title", "prediction", "trueLabel").show(n=3, truncate=False)

+------+---------------------------+----------+---------+
|userId|title                      |prediction|trueLabel|
+------+---------------------------+----------+---------+
|460   |Hudsucker Proxy, The (1994)|3.2884648 |5.0      |
|491   |Hudsucker Proxy, The (1994)|5.311088  |3.0      |
|309   |Hudsucker Proxy, The (1994)|4.0655103 |4.0      |
+------+---------------------------+----------+---------+
only showing top 3 rows



In [12]:
#Evaluate
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): nan


In [14]:
#That's occur cause there were na data
#let's clean

prediction.count()
a = prediction.count()
print("Total NA before clean: ", a)
cleanPred = prediction.dropna(how="any", subset=["prediction"])
b = cleanPred.count()
print("Total NA after clean ", b)
print("Total NA: ", a-b)

Total NA before clean:  19983
Total NA after clean  19309
Total NA:  674


In [16]:
#Evaluate Again
rmse = evaluator.evaluate(cleanPred)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 1.1992409024186217


In [39]:
# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(20)
userSubsetRecs = model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[4037, 5.9657125...|
|   463|[[968, 5.3055267]...|
|   496|[[663, 7.3805075]...|
|   148|[[446, 5.801387],...|
|   540|[[4705, 7.8119006...|
|   392|[[45666, 9.393296...|
|   243|[[4037, 5.2423387...|
|   623|[[4037, 6.02931],...|
|    31|[[1227, 6.4087696...|
|   516|[[6993, 5.710311]...|
|   580|[[7327, 5.018984]...|
|   251|[[3414, 7.2113786...|
|   451|[[2513, 8.940361]...|
|    85|[[6223, 10.426024...|
|   137|[[1050, 7.4325337...|
|    65|[[8947, 10.223124...|
|   458|[[3929, 6.04683],...|
|   481|[[83411, 6.020558...|
|    53|[[3980, 12.526106...|
|   255|[[1150, 6.79429],...|
+------+--------------------+



In [40]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
import pandas as pd

import warnings
warnings.simplefilter("ignore")

In [41]:
RecommendbyUser=userSubsetRecs.toPandas()
RecommendbyUser.head()

,userId,recommendations
0,471,"[(4037, 5.965712547302246), (188, 5.8757677078..."
1,463,"[(968, 5.3055267333984375), (446, 5.1335110664..."
2,496,"[(663, 7.380507469177246), (6541, 7.2482066154..."
3,148,"[(446, 5.801386833190918), (4037, 5.5856466293..."
4,540,"[(4705, 7.811900615692139), (48997, 7.64389991..."


In [42]:
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(20)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show(5)

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   1580|[[469, 5.9059267]...|
|   6620|[[453, 8.0317955]...|
|    471|[[395, 6.250921],...|
|   1591|[[498, 8.771437],...|
|   1342|[[174, 9.02249], ...|
+-------+--------------------+
only showing top 5 rows



In [43]:
RecommendbyMovie=movieSubSetRecs.toPandas()
RecommendbyMovie.head()

,movieId,recommendations
0,1580,"[(469, 5.905926704406738), (174, 5.65193843841..."
1,6620,"[(453, 8.031795501708984), (207, 7.01506900787..."
2,471,"[(395, 6.25092077255249), (465, 6.174275398254..."
3,1591,"[(498, 8.77143669128418), (260, 8.175416946411..."
4,1342,"[(174, 9.022489547729492), (269, 8.48479652404..."
